In [1]:
from adaptive_time.samplers import AdaptiveQuadratureSampler

import numpy as np

In [2]:
seed = 42
np.random.seed(seed)

In [3]:
horizon = 20000
dt = 1
num_steps = horizon - 1
tolerance_init = 0.01

sampler = AdaptiveQuadratureSampler(
    dt=dt,
    num_steps=num_steps,
    tolerance_init=tolerance_init,
    update_when_best=False,
)

In [4]:
def foo(xs,tol,level):
    c=int(np.floor(len(xs)/2))
    #print(xs, c)
    f = lambda xs: len(xs)*(xs[0]+xs[-1])/2 if len(xs) else 0
    if abs(f(xs) - (r:=f(xs[:c]) + f(xs[c:]))) < tol: return 1, r, [level+c]
    else: 
        x, a, cs_a = foo(xs[:c],tol/2, level)
        y, b, cs_b = foo(xs[c:],tol/2, level+c)
        return x+y+1, a+b, cs_a + cs_b + [level+c]

In [5]:
traj = np.zeros(horizon)
change_at = 19891
traj[0:change_at] = -1
trajs = [{"rews": traj}]
sample_times, total_seg, num_calls = sampler.adapt(trajs)
calls, approx_sum, ts = foo(traj, tolerance_init, 0)

print("====")
print("total_rews: {}, reward = 0 at: {}".format(sum(traj), change_at))
print("mine: num_calls {} sum {} idxes {} len {}".format(num_calls, total_seg, sample_times, len(sample_times)))
print("foo: num_calls {} sum {} idxes {} len {}".format(calls, approx_sum, np.unique(ts), len(np.unique(ts))))

0 20001 10000
0 10000 5000
10000 20001 15000
10000 15000 12500
15000 20001 17500
15000 17500 16250
17500 20001 18750
17500 18750 18125
18750 20001 19375
18750 19375 19062
19375 20001 19688
19375 19688 19531
19688 20001 19844
19688 19844 19766
19844 20001 19922
19844 19922 19883
19844 19883 19863
19883 19922 19902
19883 19902 19892
19883 19892 19887
19883 19887 19885
19887 19892 19889
19887 19889 19888
19889 19892 19890
19889 19890 19889
19890 19892 19891
19892 19902 19897
19902 19922 19912
19922 20001 19961
====
total_rews: -19891.0, reward = 0 at: 19891
mine: num_calls 29 sum -19891.0 idxes [ 5000 10000 12500 15000 16250 17500 18125 18750 19062 19375 19531 19688
 19766 19844 19863 19883 19885 19887 19888 19889 19890 19891 19892 19897
 19902 19912 19922 19961] len 28
foo: num_calls 23 sum -19891.0 idxes [ 5000 10000 12500 15000 16250 17500 18125 18750 19062 19375 19531 19687
 19765 19843 19862 19882 19886 19891 19896 19901 19911 19921 19960] len 23


In [6]:
for _ in range(1000):
    traj = np.zeros(horizon)
    change_at = np.random.randint(horizon)
    traj[0:change_at] = -1
    trajs = [{"rews": traj}]
    sample_times, total_seg, num_calls = sampler.adapt(trajs)
    calls, approx_sum, ts = foo(traj, tolerance_init, 0)

    print("====")
    print("total_rews: {}, reward = 0 at: {}".format(sum(traj), change_at))
    print("mine: num_calls {} sum {} idxes {} len {}".format(num_calls, total_seg, sample_times, len(sample_times)))
    print("foo: num_calls {} sum {} idxes {} len {}".format(calls, approx_sum, np.unique(ts), len(np.unique(ts))))

0 20001 10000
0 10000 5000
10000 20001 15000
10000 15000 12500
15000 20001 17500
15000 17500 16250
15000 16250 15625
15000 15625 15312
15625 16250 15937
15625 15937 15781
15625 15781 15703
15781 15937 15859
15781 15859 15820
15781 15820 15800
15781 15800 15790
15781 15790 15785
15790 15800 15795
15800 15820 15810
15820 15859 15839
15859 15937 15898
15937 16250 16093
16250 17500 16875
17500 20001 18750
====
total_rews: -15795.0, reward = 0 at: 15795
mine: num_calls 23 sum -15795.0 idxes [ 5000 10000 12500 15000 15312 15625 15703 15781 15785 15790 15795 15800
 15810 15820 15839 15859 15898 15937 16093 16250 16875 17500 18750] len 23
foo: num_calls 23 sum -15795.0 idxes [ 5000 10000 12500 15000 15312 15625 15703 15781 15785 15790 15795 15800
 15810 15820 15839 15859 15898 15937 16093 16250 16875 17500 18750] len 23
0 20001 10000
0 10000 5000
0 5000 2500
0 2500 1250
0 1250 625
0 625 312
625 1250 937
625 937 781
625 781 703
781 937 859
781 859 820
859 937 898
859 898 878
859 878 868
859 868

In [7]:

traj = np.linspace(-1,1,20000)
traj = traj ** 2
trajs = [{"rews": traj}]
sample_times, total_seg, num_calls = sampler.adapt(trajs)
calls, approx_sum, ts = foo(traj, tolerance_init, 0)

print("====")
print("total_rews: {}, reward = 0 at: {}".format(sum(traj), change_at))
print("mine: num_calls {} sum {} idxes {} len {}".format(num_calls, total_seg, sample_times, len(sample_times)))
print("foo: num_calls {} sum {} idxes {} len {}".format(calls, approx_sum, np.unique(ts), len(np.unique(ts))))

0 20001 10000
0 10000 5000
0 5000 2500
0 2500 1250
0 1250 625
0 625 312
0 312 156
0 156 78
0 78 39
0 39 19
0 19 9
19 39 29
39 78 58
39 58 48
58 78 68
78 156 117
78 117 97
78 97 87
97 117 107
117 156 136
117 136 126
136 156 146
156 312 234
156 234 195
156 195 175
156 175 165
175 195 185
195 234 214
195 214 204
214 234 224
234 312 273
234 273 253
234 253 243
253 273 263
273 312 292
273 292 282
292 312 302
312 625 468
312 468 390
312 390 351
312 351 331
312 331 321
331 351 341
351 390 370
351 370 360
370 390 380
390 468 429
390 429 409
390 409 399
409 429 419
429 468 448
429 448 438
448 468 458
468 625 546
468 546 507
468 507 487
468 487 477
487 507 497
507 546 526
507 526 516
526 546 536
546 625 585
546 585 565
546 565 555
565 585 575
585 625 605
585 605 595
605 625 615
625 1250 937
625 937 781
625 781 703
625 703 664
625 664 644
625 644 634
644 664 654
664 703 683
664 683 673
683 703 693
703 781 742
703 742 722
703 722 712
722 742 732
742 781 761
742 761 751
761 781 771
781 937 859
781 